# Python для сбора и обработки данных

Автор: Валентин Бирюков

# Краудсорсинг, ML и продакшон

Вспомним, что мы работали в проекте "Крутые вещи и не только", и уже успели поковыряться для нашего мелкого стартапа с аналитикой.
Мы подготовили всю продукцию, провели все сравнительные анализы, бэкендеры написали бэкенд, фронтендеры написали фронтенд... В общем запуск через пару дней, "все идет по плану".

Почти. Тут наши, допустим, продуктологи вспоминают что вообще-то мы предложили "аааабсолютно" новую возможность - обсуждать товар прям рядом с карточкой. Не только озывы после покупки, которые может оставить уже не так уж много людей - но прям в режиме живого времени комментировать классный это товар или нет (еще бы, ведь мы хотим поддерживать самую свежую базу товаров).

Но люди чуть-чуть не вписались в нашу модель. Они ведь могут оставить непойми что! Хоть мат, хоть рекламу других сервисов. Надо что-то придумывать.

Окей, базовый мат вырежем регулярками, ссылки запретим публиковать, но за всем не усмотришь, а русский язык велик и могуч! И вот вся надежда на нас, аналитиков, что мы то уж придумаем, как разобраться с такими данными.

Варианты наши одновременно обширны и скудны:
- помочь писать регулярки?
- обучить модель? но где найти разметку?
- штат модерации? нанимать людей, писать новые доступы в админку - сорвем все запуски.

И, наслушавшись лекций мы решаем пребегнуть к краудсорсингу.

В рамках данного Задания мы попробуем построить кусочки для "автоматического" (масштабируемого, дешевого) анализа текстовых комментариев. В качестве источника таковых комментариев возьмем Двач (анонимную страницу), и попробуем отмодерировать коментарии на ней

# Начальная заготовка

In [3]:
import urllib
import urllib.request
from bs4 import BeautifulSoup
import re

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

headers={'User-Agent':user_agent,} 

def make_soup(url):
    print(url)
    request= urllib.request.Request(url, None,headers)
    thepage = urllib.request.urlopen(request)
    soupdata = BeautifulSoup(thepage, "html.parser")
    return soupdata

# Научимся "обходить" двач

В рамках данной работы делать этого мы, конечно же не будем, но бдительный читатель и сам прикинет как это сделать - оно похожее на обход разных категорий в сборе данных с других площадок.

# Соберем данные из одной страницы для разметки

In [4]:
page = 'https://2ch.hk/pr/res/1647485.html'

# Подключимся к толоке

проекты/инструкции и все такое подготовим один раз в интерфейсе - как никак один раз может проще и руками.

In [5]:
TOKEN = 'MY_TOKEN'

# Загрузим все данные

# Получим результат разметки

# Проведем аггрегацию результатов